In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os

Format of the file: age_gender_ethnicity_datetime 

In [ ]:
path = "/kaggle/input/utkface-new/UTKFace/"
files = os.listdir(path)
size = len(files)
print("Total samples:",size)
print(files[0])

In [ ]:
import cv2
images = []
ages = []
genders = []
for file in files:
    image = cv2.imread(path+file,0)
    image = cv2.resize(image,dsize=(64,64))
    image = image.reshape((image.shape[0],image.shape[1],1))
    images.append(image)
    split_var = file.split('_')
    ages.append(split_var[0])
    genders.append(int(split_var[1]) )

In [ ]:
import matplotlib.pyplot as plt
x_ages = list(set(ages))
y_ages = [ages.count(i) for i in x_ages]
plt.bar(x_ages,y_ages)
plt.show()
print("Max value:",max(ages))

In [ ]:
def display(img):
    plt.imshow(img[:,:,0])
    plt.set_cmap('gray')
    plt.show()
idx = 500
sample = images[idx]
print("Gender:",genders[idx],"Age:",ages[idx])
display(sample)

In [ ]:
def age_group(age):
    if age >=0 and age < 18:
        return 1
    elif age < 30:
        return 2
    elif age < 80:
        return 3
    else:
        return 4

In [ ]:
# pre processing
target = np.zeros((size,2),dtype='float32')
features = np.zeros((size,sample.shape[0],sample.shape[1],1),dtype = 'float32')
for i in range(size):
    target[i,0] = age_group(int(ages[i])) / 4
    target[i,1] = int(genders[i])
    features[i] = images[i]
features = features / 255
display(features[550])

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2,shuffle  = True)
print("Samples in Training:",x_train.shape[0])
print("Samples in Testing:",x_test.shape[0])

In [ ]:
print("Shape of image:",sample.shape)

In [ ]:
import keras 
from keras.layers import *
from keras.models import *
from keras import backend as K

In [ ]:
inputs = Input(shape=(64,64,1))
conv1 = Conv2D(32, kernel_size=(3, 3),activation='relu')(inputs)
conv2 = Conv2D(64, kernel_size=(3, 3),activation='relu')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = Conv2D(128, kernel_size=(3, 3),activation='relu')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv3)
conv4 = Conv2D(256, kernel_size=(3,3), activation = 'relu')(pool2)
conv5 = Conv2D(256, kernel_size=(3,3), activation = 'relu')(conv4)
flat = Flatten()(conv5)

age_conv1 = Dense(256, activation='relu')(flat)
age_conv2 = Dense(128, activation='relu')(age_conv1)
age_conv3 = Dense(64, activation='relu')(age_conv2)
age_conv4 = Dense(32, activation='relu')(age_conv3)
age_model = Dense(1, activation='softmax')(age_conv4)


gender_conv1 = Dense(256, activation='relu')(flat)
gender_conv2 = Dense(128, activation='relu')(gender_conv1)
gender_conv3 = Dense(64, activation='relu')(gender_conv2)
gender_conv4 = Dense(32, activation='relu')(gender_conv3)
gender_conv5 = Dense(16, activation='relu')(gender_conv4)
gender_conv6 = Dense(8, activation='relu')(gender_conv5)
gender_model = Dense(1, activation='sigmoid')(gender_conv6)

In [ ]:
model = Model(inputs=inputs, outputs=[age_model,gender_model])
model.compile(optimizer = 'adam', loss =['mse','binary_crossentropy'],metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
import tensorflow as tf
tf.keras.utils.plot_model(model)

In [ ]:
h = model.fit(x_train,[y_train[:,0],y_train[:,1]],
              validation_data=(x_test,[y_test[:,0],y_test[:,1]]),
              epochs = 25, batch_size=128,shuffle = True)

### Visualisation 

In [ ]:
history = h
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('loss function')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
def display(img):
    plt.imshow(img[:,:,0])
    plt.set_cmap('gray')
    plt.show()
    
def age_group(age):
    if age >=0 and age < 18:
        return 1
    elif age < 30:
        return 2
    elif age < 80:
        return 3
    else:
        return 4

def get_age(distr):
    distr = distr*4
    if distr >= 0.65 and distr <= 1.4:return "0-18"
    if distr >= 1.65 and distr <= 2.4:return "19-30"
    if distr >= 2.65 and distr <= 3.4:return "31-80"
    if distr >= 3.65 and distr <= 4.4:return "80 +"
    return "Unknown"
    
def get_gender(prob):
    if prob < 0.5:return "Male"
    else: return "Female"

def get_result(sample):
    sample = sample/255
    val = model.predict( np.array([sample]) )
    age = get_age(val[0][0])
    gender = get_gender(val[1][0])
    print("Values:",val,"\nPredicted Gender:",gender,"Predicted Age:",age)
    
    
indexes = [500,59,80,2,4546,7,9,256,45]
for idx in indexes:
    sample = images[idx]
    display(sample)
    print("Actual Gender:",get_gender(genders[idx]),"Age:",ages[idx])
    res = get_result(sample)

# ****Gender Classification with Transfer Learning****

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.preprocessing.image import img_to_array,load_img
from keras import layers
from keras import optimizers
from keras.models import Sequential
from keras.utils import to_categorical
from PIL import Image
import matplotlib.pyplot as plt
import cv2

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
file_list = os.listdir(path) #Go path and list files
print("Number Of İmages: ", len(file_list))
#Gender
gender = [i.split('_')[1] for i in file_list]
gender_labels= ["Male","Female"]

gender_classes = []
y_gender= []
for i in gender:
    i= int(i)
    if i== 0:
        gender_classes.append(0)
    else:
        gender_classes.append(1)
    y_gender.append(i)
    
gender_classes= np.array(gender_classes)
print("gender_classes shape: ",gender_classes.shape) 


### Convert Images to Vector and Resize

In [ ]:
x_data=[]
for file in file_list:
    img= cv2.imread(path+'/'+file)
    img=cv2.resize(img,(48,48)) # (200,200)--->(48,48)
    x_data.append(img)
x_data= np.array(x_data)
print("x_data shape: ",x_data.shape)

### Splitting Data

In [ ]:

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_data,gender_classes,test_size=0.2,
                                               shuffle=True,random_state=42)
print("Samples in Training:",x_train.shape)
print("Samples in Testing:",x_test.shape)

y_train= to_categorical(y_train,num_classes=2) #one hot encoding
y_test= to_categorical(y_test,num_classes=2) #[1,0, ...]---> [[1,0],[0,1], ...]

print("y_train shape: ",y_train.shape)
print("y_test shape: ",y_test.shape)

input_shape= x_train.shape[1:] #we don't take the number of samples
print("input shape: ",input_shape)

## VGG-16 Architecture

In [ ]:
#%% vgg16
from keras.applications.vgg16 import VGG16 

vgg= VGG16(include_top= False,weights="imagenet",input_shape=input_shape)

#include_top=False bencause I'll modify fully connected layers
# weights='imagenet' because I want the weights used in the imagenet competition
print(vgg.summary())

### Transfer VGG-16 Model Layers to Our Model

In [ ]:
vgg_layer_list= vgg.layers #I keep the layers in the vgg_layer_list

model= Sequential() #I create my model

for layer in vgg_layer_list: 
    model.add(layer)        # I transfer vgg layers to my model

for layer in model.layers:
    layer.trainable= False  #I'll use vgg16 weights so trainable=False

model.add(layers.Flatten())
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(32, activation="relu"))
model.add(layers.Dense(2, activation="sigmoid"))

print(model.summary())

**Create Validation Dataset**

In [ ]:
x_val= x_train[12000:]
partial_x_train=x_train[:12000]

y_val= y_train[12000:]
partial_y_train=y_train[:12000]

### Compile and fit Model

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer= optimizers.RMSprop(lr=1e-5),
              metrics=["accuracy"])

In [ ]:
hist= model.fit(partial_x_train,partial_y_train,
                validation_data=(x_val,y_val),
                epochs=50,
                batch_size= 512)

## Visualisation

In [ ]:
#Train and Validation Loss
plt.plot(hist.history["loss"],label="train loss")
plt.plot(hist.history["val_loss"],label="val loss")
plt.legend()
fig1 = plt.gcf() # I create figure because ı want to save
plt.show()
plt.draw()
fig1.savefig('genderclfloss.png')

plt.figure()

##Train and Validation Accuracy
plt.plot(hist.history["accuracy"],label="train acc")
plt.plot(hist.history["val_accuracy"],label="val acc")
plt.legend()

fig2 = plt.gcf()
plt.show()
plt.draw()
fig2.savefig('genderclfacc.png')